<a href="https://colab.research.google.com/github/MaksonViini/Hackhaton-BNDES/blob/develop/BNDES_Cluster_operacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


from sklearn.cluster import KMeans

In [48]:
df = pd.read_parquet('/content/drive/Shareddrives/bndes-arquivos/operacoes.parquet')

In [49]:
df.head()

,cliente,cnpj,uf,municipio,municipio_codigo,data_da_contratacao,valor_em_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,forma_de_apoio,produto,instrumento_financeiro,inovacao,area_operacional,setor_cnae,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_do_agente_financeiro,situacao,origem,valor_contratado_reais,descricao_do_projeto,numero_do_contrato,tipo_de_garantia,tipo_de_excepcionalidade,id
0,BANCO COOPERATIVO SICOOB S.A.,**.*38.232/0001-**,SP,PEDREGULHO,3537008,2002-01-02,16000.0,16000.0,RECURSOS LIVRES - TESOURO,TAXA FIXA,"8,75",24,36,REEMBOLSAVEL,INDIRETA,BNDES AUTOMATICO,OUTROS,NAO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUARIA E PESCA,AgropecuA¡ria,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,AGROPECUARIA,AGROPECUARIA,MICRO,PRIVADA,BANCO COOPERATIVO SICOOB S.A.,02.038.232.0001-64,LIQUIDADA,AUTOMATICA,16000.0,0,0,0,0,382320001SP
1,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,TRES DE MAIO,4321808,2002-01-02,8047.0,8047.0,RECURSOS VINCULADOS - FAT DEPA“SITOS ESPECIAIS,TAXA FIXA,"4,0",0,60,REEMBOLSAVEL,INDIRETA,BNDES AUTOMATICO,PRONAF INVESTIMENTO,NAO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUARIA E PESCA,AgropecuA¡ria,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUARIA,AGROPECUARIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,AUTOMATICA,8047.0,0,0,0,0,165600001RS
2,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,8104.0,8104.0,RECURSOS VINCULADOS - FAT DEPA“SITOS ESPECIAIS,TAXA FIXA,"4,0",12,48,REEMBOLSAVEL,INDIRETA,BNDES AUTOMATICO,PRONAF INVESTIMENTO,NAO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUARIA E PESCA,AgropecuA¡ria,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUARIA,AGROPECUARIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,AUTOMATICA,8104.0,0,0,0,0,165600001RS
3,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,6304.0,6304.0,RECURSOS VINCULADOS - FAT DEPA“SITOS ESPECIAIS,TAXA FIXA,"4,0",12,48,REEMBOLSAVEL,INDIRETA,BNDES AUTOMATICO,PRONAF INVESTIMENTO,NAO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUARIA E PESCA,AgropecuA¡ria,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUARIA,AGROPECUARIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,AUTOMATICA,6304.0,0,0,0,0,165600001RS
4,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,HUMAITA,4309704,2002-01-02,15000.0,15000.0,RECURSOS VINCULADOS - FAT DEPA“SITOS ESPECIAIS,TAXA FIXA,"4,0",0,60,REEMBOLSAVEL,INDIRETA,BNDES AUTOMATICO,PRONAF INVESTIMENTO,NAO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUARIA E PESCA,AgropecuA¡ria,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUARIA,AGROPECUARIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,AUTOMATICA,15000.0,0,0,0,0,165600001RS


In [50]:
col = ['setor_cnae', 'porte_do_cliente', 'natureza_do_cliente']

# Data Transform

In [51]:
data = df[col]

In [52]:
data

,setor_cnae,porte_do_cliente,natureza_do_cliente
0,AGROPECUARIA E PESCA,MICRO,PRIVADA
1,AGROPECUARIA E PESCA,MICRO,PRIVADA
2,AGROPECUARIA E PESCA,MICRO,PRIVADA
3,AGROPECUARIA E PESCA,MICRO,PRIVADA
4,AGROPECUARIA E PESCA,MICRO,PRIVADA
...,...,...,...
1928488,INDUSTRIA DE TRANSFORMAAAO,GRANDE,PRIVADA
1928489,INDUSTRIA DE TRANSFORMAAAO,GRANDE,PRIVADA
1928490,INDUSTRIA DE TRANSFORMAAAO,GRANDE,PRIVADA
1928491,INDUSTRIA DE TRANSFORMAAAO,GRANDE,PRIVADA


In [53]:
df['natureza_do_cliente'].unique()

array(['PRIVADA', 'ADMINISTRAAAO PAsBLICA DIRETA - GOVERNO MUNICIPAL',
       'PAsBLICA INDIRETA',
       'ADMINISTRAAAO PAsBLICA DIRETA - GOVERNO ESTADUAL',
       'ADMINISTRAAAO PAsBLICA DIRETA - GOVERNO FEDERAL'], dtype=object)

In [54]:
func_porte = lambda x: (0 if x == 'MICRO' else
                        1 if x == 'PEQUENA' else
                        2 if x == 'GRANDE' else
                        3)

func_setor = lambda x: (0 if x == 'AGROPECUARIA E PESCA' else 
                        1 if x == 'COMERCIO E SERVICOS' else 
                        2 if x == 'INDUSTRIA DE TRANSFORMAAAO' else 
                        3)

func_natureza_cliente = lambda x: (0 if x == 'PRIVADA' else
                                   1 if x == 'ADMINISTRAAAO PAsBLICA DIRETA - GOVERNO MUNICIPAL' else
                                   2 if x == 'PAsBLICA INDIRETA' else
                                   3 if x == 'ADMINISTRAAAO PAsBLICA DIRETA - GOVERNO ESTADUAL' else 
                                   4)

In [55]:
data['porte_do_cliente'] = data['porte_do_cliente'].apply(func_porte)
data['setor_cnae'] = data['setor_cnae'].apply(func_setor)
data['natureza_do_cliente'] = data['natureza_do_cliente'].apply(func_natureza_cliente)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [56]:
data.head()

,setor_cnae,porte_do_cliente,natureza_do_cliente
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [57]:
data['setor_cnae'].unique()

array([0, 1, 2, 3])

# Model 

In [58]:
X = data.values

In [59]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(X)

In [60]:
X_clusted = kmeans.predict(X)

In [61]:
data['label'] = X_clusted

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
data.head()

,setor_cnae,porte_do_cliente,natureza_do_cliente,label
0,0,0,0,3
1,0,0,0,3
2,0,0,0,3
3,0,0,0,3
4,0,0,0,3


In [63]:
 data.groupby('setor_cnae')['label'].value_counts()

setor_cnae  label
0           3        265377
            1         28494
1           0        567296
            1        412486
            2        333976
2           1        172384
            0         69186
            2         64814
3           1          6051
            2          4312
            0          4117
Name: label, dtype: int64

In [64]:
data.to_csv('/content/drive/Shareddrives/bndes-arquivos/operacoes_cluster.csv', index=False)